# Exercises due by EOD 2017.11.09

## goal

in this homework assignment we will focus on `aws lambda` function deployment.

## method of delivery

as mentioned in our first lecture, the method of delivery may change from assignment to assignment. we will include this section in every assignment to provide an overview of how we expect homework results to be submitted, and to provide background notes or explanations for "new" delivery concepts or methods.

this week you will be submitting the results of your homework via upload to two different `s3` buckets. the first is one I created for you and will be used for question 1, and the second is the same homework bucket you have been using now for several weeks.

summary:

| exercise | deliverable                               | method of delivery                                  |
|----------|-------------------------------------------|-----------------------------------------------------|
| 1        | deployment archive `airportweather.zip`   | upload to our `s3` bucket: `2017.fall.gu511.{GUID}` |
| 1        | deployment command `how_i_deployed_it.sh` | upload to our `s3` bucket: `2017.fall.gu511.{GUID}` |
| 2        | a test message in a database              | `INSERT`ed into a shared `aws rds` database         |

# exercise 1: create a `lambda` function from a deployment package

let's create a `zip` archive containing all the material we need to execute a `lambda` function.

*note*: for this assignment, you must perform all of your work in a `linux` envrionment. the `python` code we will use will be OS-agnostic but the external library (`requests`) may not be. your mac or windows laptops are not guaranteed to produce the same results and I haven't tested or verified whether they would.

it's also worth noting: the `lambda` function code will run on the `amazon linux ami`, so you may want to consider creating a new `ec2` instance using that `ami` for your work (though I can confirm that the `ubuntu ami` we used for our `ec2` instances *will* work in this instance).


## get the `python` files

create an isolated directory on your `linux` instance with the name `airportweather`, and download into this new directory the following two `python` files:

+ `airportweather.py`: a file implementing a simple `requests HTTP` call to our dca weather api
    + https://s3.amazonaws.com/shared.rzl.gu511.com/airportweather.py 
+ `utils.py`: generalized / abstracted utility functions for defining future `aws lambda` functions
    + https://s3.amazonaws.com/shared.rzl.gu511.com/utils.py

using a `python 3.6` environment with the `requests` library installed (via `conda install requests`), verify that you have correctly downloaded the file by running

```bash
python airportweather.py
```

and verifying that the `json` output of the dca weather api is printed to the screen. you should see something like

```json
{'statusCode': '200', 'body': {'delay': 'false', 'IATA': 'DCA', 'state': 'District of Columbia', 'name': 'Ronald Reagan Washington National', 'weather': {'visibility': 10.0, 'weather': 'Overcast', 'meta': {'credit': "NOAA's National Weather Service", 'updated': '4:52 PM Local', 'url': 'http://weather.gov/'}, 'temp': '61.0 F (16.1 C)', 'wind': 'Southwest at 5.8mph'}, 'ICAO': 'KDCA', 'city': 'Washington', 'status': {'reason': 'No known delays for this airport.', 'closureBegin': '', 'endTime': '', 'minDelay': '', 'avgDelay': '', 'maxDelay': '', 'closureEnd': '', 'trend': '', 'type': ''}}, 'headers': {'Content-Type': 'application/json'}}
```


## get a copy of the `requests` library

if we were to just `zip` up the two `python` files right now and deploy those, our script would fail. why?

we use the `requests` library, and our `python 3.6` runtime **does not come with** that library. `requests`, in turn, depends on several *other* non-standard libraries. we need to include them all in our deployment package

one standard way of building up this deployment package is to do the following:

1. create a brand new `conda` environment named `airportweather` with `python 3.6`
2. `activate` that environment
3. `install requests` (and automatically its dependencies)
    1. this last command will install many packages -- make note of them!

for example, when I installed in `ubuntu`:

```bash
asn1crypto:   0.22.0-py36h265ca7c_1
cffi:         1.10.0-py36had8d393_1
chardet:      3.0.4-py36h0f667ec_1 
cryptography: 2.0.3-py36ha225213_1 
idna:         2.6-py36h82fb2a8_1   
pycparser:    2.18-py36hf9f622e_1  
pyopenssl:    17.2.0-py36h5cc804b_0
pysocks:      1.6.7-py36hd97a5b1_1 
requests:     2.18.4-py36he2e5f8d_1
six:          1.11.0-py36h372c433_1
urllib3:      1.22-py36hbe7ace6_0  
```

we need to provide these extra libraries, and the way we will do that is by taking the files that define it (in our `conda` environment) and copying them into the same folder our other `python` code lives in -- they will then be included in our deployment archive.

you may already know where those files are located, but if not, you can find out. in a `python` session, execute the following:

```python
import requests
print(requests.__file__)
```

the result should be something like

```bash
/path/to/conda/env/airportweather/lib/python3.6/site-packages/requests/__init__.py
```

many packages and files live in `/path/to/conda/env/airportweather/lib/python3.6/site-packages/` -- these files form the basis of our `python` environment.

copy each of them to the `airportweather` directory:

```bash
# this assumes your working directory (pwd) is your `airportweather` directory
cp -r /path/to/conda/env/airportweather/lib/python3.6/site-packages/* .
```

this means that your `airportweather` directory should have many items in it:

1. `airportweather.py`
2. `utils.py`
3. the `requests` folder
4. several other `python` library folders and files

*note*: many of these files and folders are unecessary. The `pip`, `wheel`, `certifi`, `OpenSSL`, `pkg_resources`, and `setuptools` directories were all included *before* we `install`ed `requests` -- they are *built-in* packages, so they will already exist in our `aws lambda` runtime. in a true deployment setting, we would prune these libraries before moving on. for now, though, let's just leave them in for simplicity's sake


## `zip` it

zip the contents of `airportweather` into a `zip` archive named `airportweather.zip`:

```bash
zip -r9 airportweather.zip ./*
```

now, right besides the two `python` library folders and two `python` files, you should *also* have one `zip` archive.


## deploy it

using the `aws lambda create-function` subcommand, *deploy* this `zip` archive as a `lambda` function named `airportweather`. create a new `iam role` for this or re-use one from previous homeworks or the lecture -- that much is up to you! you will need to determine values for the following command line flags:

+ `function-name`
+ `zip-file`
+ `role`
+ `handler`
+ `runtime`

write whatever `aws` command(s) you use to deploy this `lambda` function into a plain-text shell script file `how_i_deployed_it.sh`


## test it

this command should have created a new `lambda` function for you named `airportweather`. test that `lambda` function from the command line using the `aws lambda invoke` command:

```bash
aws lambda invoke  \
    --function-name airportweather  \
    --payload '{"airportcode": "DCA"}'  \
    /tmp/dcaweather.json
```

this command should print to the terminal

```bash
{
    "StatusCode": 200
}
```

and the contents of `/tmp/dcaweather.json` should be the familiar DCA weather report in `json` form.

if you see the same thing, congratulations -- you've deployed a complex `lambda` function!


##### upload `airportweather.zip` and `how_i_deployed_it.sh` to the new `2017.fall.gu511.{GUID} s3` bucket (which we own but on which you should have full permissions). make sure you add flags `--acl bucket-owner-full-control`

# exercise 2: connect and write to a `postgres` database

we have set up a shared `postgres` database with the following connection details:

| parameter  | value                                                       |
|------------|-------------------------------------------------------------|
| `host`     | `rzl-gu511-shared.cdmknaubrmaw.us-east-1.rds.amazonaws.com` |
| `port`     | `5432`                                                      |
| `username` | `gu511shared`                                               |
| `db`       | `gu511`                                                     |
| `password` | written to `s3://2017.fall.gu511.{GUID}/dbpw.txt`           |

we would like for you to install the `psql` client on your `ec2 ubuntu` instances, use it to connect to this shared database, and `insert` a value into a shared table.


## install `psql`

on your `ec2 ubuntu` instances, install `psql` using the command

```bash
sudo apt install postgresql-client
```

you can verify that your installation worked be checking

```bash
which psql
```


## connect to the database

using the values we provided in the table above, the course notes, and `man psql`, figure out how to connect to our shared database with the `psql` command


## `insert` a new record

the following command will insert a record into a shared database we've created called `gu511_shared_table`. the table has two columns:

1. `guid`: your georgetown user id
2. `message`: a test message to verify you were able to successfully insert your record

once you've successfully connect to the `postgres` database, insert a record with the following query (replacing `MY_GUID` and `MY_MESSAGE` with real values, of course):

```sql
INSERT INTO gu511_shared_table (guid, message) VALUES ('MY_GUID', 'MY_MESSAGE');
```

*note*: the single quotes in `'MY_GUID', 'MY_MESSAGE'` are required. they indicate that the values being passed in are strings, which is the datatype of those two coumns in our table


##### the submission is the record in the shared database -- we will check results by running `SELECT * FROM gu511_shared_table;`